In [1]:
import os
import cuid

In [2]:
os.chdir("./../.")

In [3]:
import dotenv
from pymongo import MongoClient

In [4]:
from app import utils

In [ ]:
dotenv.load_dotenv()

In [6]:
mongodb_url = os.environ.get("MONGODB_URL")

In [7]:
mongodb_client = MongoClient(mongodb_url)

In [8]:
db = mongodb_client['newsclocker']

### (1) Read Input Prompt


In [9]:
prompt_paths = ["./prompts/prompt_investment.json",
                "./prompts/prompt_2.json", "./prompts/prompt_1.json"]

In [10]:
prompt_list = []
for path in prompt_paths:
    prompt_list.extend(utils.read_json_file(path))

In [11]:
# prompt_list

### (2) Ingest Category


In [12]:
unique_categories = list(set([p['category'] for p in prompt_list]))

In [13]:
existing_categories = set([item['category']
                           for item in db['PromptCategory'].find()])

In [14]:
new_categories = [
    {"category": category, "_id": cuid.cuid()} for category in unique_categories if category not in existing_categories]

In [15]:
if new_categories:
    db['PromptCategory'].insert_many(new_categories)

### (3) Ingest the prompt


In [16]:
userId = "public"

In [17]:
input_prompt_docs = []

for prompt in prompt_list:
    prompt.pop("id")
    prompt['userId'] = "public"
    prompt['share'] = True
    prompt['_id'] = cuid.cuid()
    input_prompt_docs.append(prompt)

In [ ]:
input_prompt_docs[0]

In [ ]:
db['Prompt'].insert_many(input_prompt_docs)